In [2]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import progressbar

In [5]:
df = pd.read_csv('introdf.csv')
df.head()

df['abstract'][0]

"Many investigations of sensory processing are based on the principle that the capacity to discriminate between two sensory stimuli must be based upon the difference between their neural representations. Thus, identifying how manipulations of a stimulus alter its percept can help elucidate the neural representation. The present work addresses the nature of neural coding in the somatosensory system: we have conducted psychophysical experiments to identify which features of a vibrotactile stimulus are extracted by the somatosensory system to determine its frequency, and which regions in the somatosensory cortical network are involved in this process.Early investigations focused on the role of neurons in subcortical stations and primary somatosensory cortex (SI) in coding low frequency “flutter” vibrations (below 50 Hz) [1]–[3], while more recent work has emphasized the role of cortical areas “downstream” from SI, such as the second somatosensory cortex (SII) and regions of frontal cortex

In [6]:
#sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=True, sublinear_tf=True)
#doc_mat = sklearn_tfidf.fit_transform(df['abstract'].values.astype('U'))

In [7]:
docs = []
bar = progressbar.ProgressBar()

for i in bar(range(len(df.index))):
    if str(df.iloc[i]['abstract']) != 'nan':
        docs.append(df.iloc[i]['abstract'])
    else:
        continue


100% (10000 of 10000) |###################| Elapsed Time: 0:00:02 Time: 0:00:02


In [8]:
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

In [9]:
# list for tokenized documents in loop
texts = []

# loop through document list
for i in docs:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

In [10]:
len(texts)

9726

In [11]:
dictionary = corpora.Dictionary(texts)
print(dictionary)

Dictionary(81966 unique tokens: ['1', '10', '11', '12', '13']...)


In [12]:
corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = models.TfidfModel(corpus)

In [9]:
#ldamodel = models.LdaModel(corpus, num_topics=10, id2word = dictionary)
#bow- not tf-idf

In [13]:
ldamodel2 = models.LdaModel(tfidf[corpus], num_topics=10, id2word = dictionary)
## save model - look at docs
## passes=20

In [14]:
ldamodel2.show_topics()

[(0,
  '0.001*"mrsa" + 0.001*"raft" + 0.001*"allergen" + 0.001*"obp" + 0.001*"dysbindin" + 0.000*"song" + 0.000*"exosom" + 0.000*"flotillin" + 0.000*"methicillin" + 0.000*"tcf7l2"'),
 (1,
  '0.004*"mirna" + 0.001*"mir" + 0.001*"malaria" + 0.001*"obes" + 0.001*"sleep" + 0.001*"speci" + 0.001*"rna" + 0.001*"bat" + 0.001*"food" + 0.001*"pylori"'),
 (2,
  '0.001*"chikv" + 0.000*"wolbachia" + 0.000*"hmgb1" + 0.000*"ssri" + 0.000*"crf" + 0.000*"albopictu" + 0.000*"geneid" + 0.000*"qd" + 0.000*"nhej" + 0.000*"t3ss"'),
 (3,
  '0.002*"task" + 0.002*"visual" + 0.001*"memori" + 0.001*"emot" + 0.001*"face" + 0.001*"learn" + 0.001*"object" + 0.001*"percept" + 0.001*"stimuli" + 0.001*"cortex"'),
 (4,
  '0.001*"mtorc1" + 0.001*"amd" + 0.001*"npi" + 0.001*"vdr" + 0.001*"imput" + 0.001*"tsc2" + 0.000*"fto" + 0.000*"schistosomiasi" + 0.000*"fmr1" + 0.000*"autophagi"'),
 (5,
  '0.001*"gene" + 0.001*"cell" + 0.001*"methyl" + 0.001*"es" + 0.001*"dna" + 0.001*"express" + 0.001*"drug" + 0.001*"protein" + 0.0

In [15]:
ldamodel2.get_topic_terms(9)

[(6786, 0.0009708681),
 (4162, 0.00081612234),
 (14679, 0.00077345036),
 (4547, 0.00076299103),
 (2959, 0.00074732152),
 (2508, 0.00064930454),
 (10525, 0.00064542581),
 (4439, 0.00064032339),
 (150, 0.00058761198),
 (5509, 0.00058517326)]

In [16]:
ldamodel2.get_document_topics(corpus[1])

[(1, 0.0293251), (3, 0.020236894), (4, 0.03966368), (6, 0.90103412)]

In [17]:
doc1tops = ldamodel2.show_topic(7)
doc1tops

[('salmonella', 0.00082862494),
 ('t1d', 0.00075817679),
 ('msi', 0.00056358147),
 ('typhimurium', 0.00050283695),
 ('typhi', 0.00049295975),
 ('cilia', 0.00049181603),
 ('asd', 0.00046959799),
 ('autism', 0.00046411535),
 ('t2dm', 0.00044025693),
 ('cin', 0.00041626647)]

In [18]:
tops = [i[0] for i in doc1tops]

In [19]:
toptops = []
bar = progressbar.ProgressBar()

for i in bar(range(0,len(texts))):
    top = ldamodel2.show_topic(ldamodel2.get_document_topics(corpus[i])[0][0])
    toptops.append([j[0] for j in top])

100% (9726 of 9726) |#####################| Elapsed Time: 0:00:33 Time: 0:00:33


In [20]:
print(toptops[1])

['mirna', 'mir', 'malaria', 'obes', 'sleep', 'speci', 'rna', 'bat', 'food', 'pylori']


In [ ]:
#Association -- just overlapping topics? Or combination of topics shown together? Or words
#across topics that are shown together? 

#Does the presence of a topic predict the presence of another topic?
#40+ topics
#Top topics -- above threshold (0.05) 

#pairwise mutual information among topics?? Raf
# contiguity rather than similarity (Royal navy and Georgia timber)

## Doc2Vec

In [32]:
from gensim.models.doc2vec import Doc2Vec,TaggedLineDocument
import gzip,os,glob
import numpy as np
from nltk.tokenize.moses import MosesDetokenizer

In [33]:
detokenizer = MosesDetokenizer()
file = open('processed_docs.txt', 'w')
bar = progressbar.ProgressBar()
  
for i in bar(range(0,len(texts))):
    detok_str = detokenizer.detokenize(texts[i], return_str=True)
    file.write("%s\n" % detok_str)


100% (9726 of 9726) |#####################| Elapsed Time: 0:01:45 Time: 0:01:45


In [37]:
file.close()

In [38]:
# Generate a list of tagged line documents, one per line from the file generated above
documents = [doc for doc in TaggedLineDocument('processed_docs.txt')]

In [41]:
# Now just run the model. A simple little one-liner. Be sure to take advantages of the the `workers` option, which parallelizes the model automagically
model = Doc2Vec(documents, size=200, window=5, min_count=5,workers=24)

In [42]:
# the resulting feature vectors are stored in `model.docvecs.doctag_syn0`, so let's just save that numpy array
np.save('features-w2v-200.npy',model.docvecs.doctag_syn0)

In [46]:
doc_id = np.random.randint(model.docvecs.count) 
sims = model.docvecs.most_similar(doc_id, topn=model.docvecs.count)  # get *all* similar documents
print(u'TARGET (%d): «%s»\n' % (doc_id, ' '.join(documents[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(documents[sims[index][0]].words)))

TARGET (1094): «dietari selenium potent cancer prevent activ 1 selenium contain protein selenoprotein 2 3 refer therein low molecular weight selenium contain compound selenocompound 2 refer therein implic activ major role selenium provid health benefit like action selenoprotein 1 thioredoxin reductas 1 tr1 one 24 known selenoprotein rodent 4 major antioxid redox regul mammalian cell 5 6 refer therein essenti role mammalian develop 7 howev enzym appear oppos effect cancer develop implic cancer prevent 8 cancer promot 9 12 exampl tr1 support p53 function tumor suppressor activ target carcinogen electrophil compound argu role cancer prevent 13 altern tr1 overexpress mani cancer cell 9 12 inhibit varieti potent cancer drug alter cancer relat properti numer tumor malign cell suggest enzym target cancer therapi 9 12 14 15 clear whether tr1 cancer prevent cancer promot properti exert greater influenc cancer whether contrast effect oper simultan specif differ stage cancer develop properti util